フィールドの型を上書きできるか？というアプローチ (できなかった)

In [ ]:
from typing import TypedDict

from prisma import Prisma
from prisma.bases import BaseChild, BaseParent

In [2]:
class ChildMeta(TypedDict):
    pass


class Child(BaseChild):
    meta: ChildMeta


class ParentMeta(TypedDict):
    pass


class Parent(BaseParent):
    children: list[Child]
    meta: ParentMeta

In [3]:
async with Prisma(auto_register=True):

    # INSERT: ParentCreateInput は書き換えれないのでエラーが出てしまう...
    await Parent.prisma().create(
        {
            "name": "foo",
            "meta": ParentMeta(),
            "children": {
                "create": [
                    {
                        "name": "bar",
                        "meta": ChildMeta(),
                    }
                ]
            },
        }
    )

    # SELECT: こちらもエラーが出る
    parent = await Parent.prisma().find_first_or_raise(
        order={"created_at": "desc"},
        include={"children": True},
    )
    print(type(parent.meta))

DataError: Unable to match input value to any allowed input type for the field. Parse errors: [Unable to match input value to any allowed input type for the field. Parse errors: [Invalid argument type. `meta` should be of any of the following types: `JsonNullValueInput`, Invalid argument type. `meta` should be of any of the following types: `Json`], Unable to match input value to any allowed input type for the field. Parse errors: [Invalid argument type. `meta` should be of any of the following types: `JsonNullValueInput`, Invalid argument type. `meta` should be of any of the following types: `Json`]]